In [19]:
id_region={"Beni-Mellal-Khenifra":5,"Casablanca-Settat":6,"Draa-Tafilalet":8,"Eddakhla-Oued-Eddahab":12,
           "Fes-Meknes":3,"Guelmim-Oued-Noun":10,"Laayoune-Sakia-El-Hamra":11,"Marrakech-Safi":7,"Oriental":2,
           "Rabat-Sale-Kenitra":4,"Souss-Massa":9,"Tanger-Tetouan-Al-Hoceima":1}
nbr_region={"Beni-Mellal-Khenifra":5,"Casablanca-Settat":9,"Draa-Tafilalet":5,"Eddakhla-Oued-Eddahab":2,
           "Fes-Meknes":9,"Guelmim-Oued-Noun":4,"Laayoune-Sakia-El-Hamra":4,"Marrakech-Safi":8,"Oriental":8,
           "Rabat-Sale-Kenitra":7,"Souss-Massa":6,"Tanger-Tetouan-Al-Hoceima":8}
import json
path="C:/Users/pc/Desktop/MoroccoMap/prov_id.json"
f = open(path)
id_province = json.load(f)
f.close()

In [31]:
import numpy as np
nbr=np.array(list(nbr_region.values()))
nbr=[0]+list(nbr.cumsum())
nbr
dicti=dict()
region=list(id_region.keys())
for i in range(len(id_region)):
    prv=list(id_province.keys())[nbr[i]:nbr[i+1]]
    code=list(id_province.values())[nbr[i]:nbr[i+1]]
    dicti[region[i]]=dict(zip(prv,code))

In [65]:
def texTocoord(text):
    txt=text.split(" ")
    x,y=[],[]
    i=0
    while i< (len(txt)-1):
        x.append(txt[i])
        y.append(txt[i+1])
        i+=2
    return x,y

In [88]:
def getData(data_dict,id_r,id_p):
    idx="RGPH:communes_reg"+str(id_r)+"_prov"+str(id_p)
    province=data_dict['wfs:FeatureCollection']['gml:featureMembers'][idx]
    libelle=["RGPH:prov_fr","RGPH:prov_ar","RGPH:code_prov","RGPH:com_fr","RGPH:cod_com","RGPH:cercle_fr",
         "RGPH:cod_cercle","RGPH:com_fr","RGPH:cod_com",
         'RGPH:libelle_ar', 'RGPH:libelle_fr' ,'RGPH:region_fr', 'RGPH:region_ar','RGPH:code_reg', 'RGPH:code_prov']
    libelle_c=[x.replace("RGPH:","") for x in libelle]+["Data"]
    data=[]
    for i in range(len(province)):
        prv=[]
        for lib in libelle :
            prv.append(province[i][lib])
        try :
            x,y=texTocoord(province[i]['RGPH:the_geom']['gml:MultiSurface']['gml:surfaceMember']['gml:Polygon']['gml:exterior']['gml:LinearRing']['gml:posList'])
        except:
            x,y=texTocoord(province[i]['RGPH:the_geom']['gml:MultiSurface']['gml:surfaceMember'][0]['gml:Polygon']['gml:exterior']['gml:LinearRing']['gml:posList'])
        prv.append((x,y))
        prv=dict(zip(libelle_c,prv))
        data.append(prv)
    return data

In [93]:
import xmltodict
def xmlTojson(id_region):
    for region in list(id_region.keys()):
        for province in list(dicti[region].keys()):
            path="C:/Users/pc/Desktop/MoroccoMap/"+region+"/"+province+".xml"
            with open(path,encoding='utf-8') as xml_file:
                data_dict = xmltodict.parse(xml_file.read())
            xml_file.close()
            vals=getData(data_dict,id_region[region],dicti[region][province])
            data={"Region":region,"Province":province,"Data":vals}
            path=path.replace('xml','json')
            with open(path, "w") as outfile:
                json.dump(data, outfile)
            print("==>",sep='',end='',flush=True)
        print("Export json: "+ region)
#xmlTojson(id_region)

In [94]:
%%time
xmlTojson(id_region)

==>==>==>==>==>Export json: Beni-Mellal-Khenifra
==>==>==>==>==>==>==>==>==>Export json: Casablanca-Settat
==>==>==>==>==>Export json: Draa-Tafilalet
==>==>Export json: Eddakhla-Oued-Eddahab
==>==>==>==>==>==>==>==>==>Export json: Fes-Meknes
==>==>==>==>Export json: Guelmim-Oued-Noun
==>==>==>==>Export json: Laayoune-Sakia-El-Hamra
==>==>==>==>==>==>==>==>Export json: Marrakech-Safi
==>==>==>==>==>==>==>==>Export json: Oriental
==>==>==>==>==>==>==>Export json: Rabat-Sale-Kenitra
==>==>==>==>==>==>Export json: Souss-Massa
==>==>==>==>==>==>==>==>Export json: Tanger-Tetouan-Al-Hoceima
Wall time: 25.2 s


In [1]:
import xmltodict
with open("C:/Users/pc/Desktop/MoroccoMap/Maroc.xml",encoding='utf-8') as xml_file:
    data_dict = xmltodict.parse(xml_file.read())
xml_file.close()

In [1]:
#Extract from Maroc.xml
def xmlTojson2():
        path="C:/Users/pc/Desktop/MoroccoMap/Maroc.xml"
        with open(path,encoding='utf-8') as xml_file:
            data_dict = xmltodict.parse(xml_file.read())
        xml_file.close()
        region=data_dict['wfs:FeatureCollection']['gml:featureMembers']['RGPH:regions']
        libelle=['RGPH:libelle_fr','RGPH:libelle_ar','RGPH:code_reg']
        libelle_c=[x.replace('RGPH:','') for x in libelle]+['Coordinates']
        data=dict()
        for i in range(len(region)):
            reg=[]
            for lib in libelle:
                reg.append(region[i][lib])
            x,y=texTocoord(region[i]['RGPH:the_geom']['gml:MultiSurface']['gml:surfaceMember']['gml:Polygon']['gml:exterior']['gml:LinearRing']['gml:posList'])
            reg.append((x,y))
            reg=dict(zip(libelle_c,reg))
            data[reg['libelle_fr'].replace("é","e").replace("è","e").replace(" ","-").replace("â",'a')]=reg
        path=path.replace('xml','json')
        with open(path, "w") as outfile:
            json.dump(data, outfile)
        print("JSON exported ")
xmlTojson2()